In [1]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -U \
langchain-community==0.3.5 \
pymupdf==1.24.13 \
faiss-gpu==1.7.2 \
SQLAlchemy==2.0.35 \
dataclasses-json==0.6.7 \
httpx-sse==0.4.0 \
pydantic-settings==2.6.1 \
marshmallow==3.23.1 \
python-dotenv==1.0.1 \
typing-inspect==0.9.0 \
mypy-extensions==1.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 27.3 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.2.10
    Uninstalling langsmith-0.2.10:
      Successfully uninstalled langsmith-0.2.10


In [4]:
import torch
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# PDF 로드
loader = PyMuPDFLoader("/content/drive/MyDrive/Colab Notebooks/pdf/BBS_202402151054353090.pdf")
documents = loader.load()

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

# Document 객체에서 텍스트 내용 추출
text_contents = [doc.page_content for doc in texts]

# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# 벡터 스토어 설정
vectorstore = FAISS.from_texts(text_contents, embeddings)

# BGE Reranker 모델 설정
model_name = "BAAI/bge-reranker-v2-m3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def rerank_documents(query, docs, top_k=5):
    pairs = [[query, doc.page_content] for doc in docs]
    with torch.no_grad():
        inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors="pt", max_length=512)
        scores = model(**inputs).logits.squeeze(-1)
    ranked_indices = scores.argsort(descending=True)
    reranked_docs = [(docs[i], scores[i].item()) for i in ranked_indices[:top_k]]
    return reranked_docs

# 리트리버 설정
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

# 쿼리 실행
query = "국내 고령화 전망에 대해 알려주세요"

# 기본 검색 결과 출력
print("## 기본 검색 결과")
base_docs = base_retriever.invoke(query)
for i, doc in enumerate(base_docs[:5]):
    print(f"문서 {i+1}:")
    print(doc.page_content[:100] + "...\n")

# 리랭킹 결과 출력
print("\n## 리랭킹 결과")
reranked_docs = rerank_documents(query, base_docs)
for i, (doc, score) in enumerate(reranked_docs):
    print(f"문서 {i+1} (점수: {score:.4f}):")
    print(doc.page_content[:100] + "...\n")

<ipython-input-4-3a41f27e756c>:20: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

## 기본 검색 결과
문서 1:
공공기관의 돌봄서비스 제공에 활용할 수 있는 기술개발이 필요
 신체 및 정서 변화를 겪는 고령층을 위한 다양한 기술과 서비스는 글로벌 사회가 직면한 
초고령 사회의 문제점을 해결...

문서 2:
고령화시대 해결을 위한 기술개발
- 19 -
◎ 돌봄 분야
 (한국, 스마트요양원) 2020년부터 부산에서 요양원을 디지털 전환하는 ‘스마트 요양원 사업’을 
진행하여 현장에 적...

문서 3:
TIPA 이슈 리포트
Vol. 6
- 14 -
Ⅲ. 고령화 기술·제품 동향
◎ 신체활동/이동 보조 분야
 (한국, 스마트 지팡기) 말하는 스마트 지팡기 ‘톡톡스틱’은 기존 지팡이...

문서 4:
고령화시대 해결을 위한 기술개발
- 17 -
◎ 정서/인지 케어 분야
 (한국, 인지치료 로봇) ICT 인지중재 프로그램이 탑재된 앵무새 로봇 ‘피오’
▪ 경도인지장애 및 초기치...

문서 5:
정기적으로 전달하고 응답내용을 분석하여 위험에 처하거나 특별한 주의가 필요하다고 인식되면 응급 
알람 메시지를 가족이나 의사에게 전달
▪ 시니어의 건강상태가 좋지않아 즉각적인 도움...


## 리랭킹 결과
문서 1 (점수: 3.0630):
고령화시대 해결을 위한 기술개발
- 3 -
Ⅰ. 고령화시대의 도래
1
국내외 고령화 현황 및 전망
 고령화(高齡化)란 평균 수명의 증가에 따라 총 인구 중 차지하는 노인의 인구비...

문서 2 (점수: -1.5022):
고령화시대 해결을 위한 기술개발
- 7 -
 정부는 고령친화 우수사업자·고령친화 우수제품 지정제도 도입으로 안심하고 고령친화사업의 
용품을 이용할 수 있도록 하고 있으나 고령친화...

문서 3 (점수: -2.1617):
고령화시대 해결을 위한 기술개발
- 21 -
Ⅴ. 결론 및 시사점
 현재 고령친화산업은 사물인터넷(IoT), 모바일과 인공지능의 발달과 비대면 서비스의 확대로 
국내외 관심이 더...

문서 4 (점수: -2.7052):
TIPA 이슈 리포트
Vol. 6
- 20 -
Ⅳ.